In [1]:
library(reticulate)
library(plyr)
library(tidyr)
library(dplyr)
library(DALEX)
library(ggplot2)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:plyr’:

    arrange, count, desc, failwith, id, mutate, rename, summarise,
    summarize


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Welcome to DALEX (version: 2.2.0).
Find examples and detailed introduction at: http://ema.drwhy.ai/
Additional features will be available after installation of: ggpubr.
Use 'install_dependencies()' to get all suggested dependencies


Attaching package: ‘DALEX’


The following object is masked from ‘package:dplyr’:

    explain




In [2]:
library(ingredients)


Attaching package: ‘ingredients’


The following object is masked from ‘package:DALEX’:

    feature_importance




In [3]:
use_condaenv('analiza_dyfuzji', required = TRUE)
source_python('utils.py')
os <- import('os')

In [4]:
model <- load_model('Models/model_new.sav')

In [5]:
data <- load_data('Data/characteristics/characteristics_new.csv', 0.3)

In [6]:
class_vector <- unlist(CLASS_MAPPING, use.names = FALSE)

In [7]:
x_train <- data[[1]]
x_test <- data[[2]]
y_train <- data[[3]]
y_test <- data[[4]]

In [8]:
dim(x_test)

[1] 18908    43

### Create explainer

In [9]:
explainer <- explain(model,
       data=x_train,
       y=y_train,
       label='Model',
       predict_function=custom_predict,
       residual_function=custom_residual_function)

Preparation of a new explainer is initiated
  -> model label       :  Model 
  -> data              :  44117  rows  43  cols 
  -> target variable   :  44117  values 
  -> predict function  : 'predict_function' is not a 'function' class object! (  WARNING  )
  -> predict function  :  custom_predict 
  -> predicted values  :  No value for predict function target column. (  default  )
  -> model_info        :  package Model of class: sklearn.ensemble._gb.GradientBoostingClassifier package unrecognized , ver. Unknown , task regression (  default  ) 
  -> predicted values  :  predict function returns multiple columns:  5  (  default  ) 
  -> residual function : 'residual_function' is not a 'function' class object! (  WARNING  )
  -> residual function :  custom_residual_function 
  -> residuals         :  the residual_function returns an error when executed (  WARNING  ) 
  A new explainer has been created!  


In [10]:
THEME_CUSTOM <- theme(plot.title = element_text(size = 20, hjust = 0.5, face = "bold"),
                      text = element_text(face = "bold"),
                      legend.text = element_text(size = 11, face = "bold"),
                      legend.position = "bottom",
                      legend.key.width = unit(1, "cm"),
                      strip.background = element_rect(color = "black", fill = "white", size = 1, linetype = "solid"))


In [26]:
create_bd_plot <- function(explainer, x){
    bd <- predict_parts(explainer, x, type='break_down', N=500)
    bd$label <- mapvalues(bd$label, from=c(unique(bd$label)), to=unlist(CLASS_MAPPING, use.names = FALSE))
    bd
}

save_plot <- function(path, bd, height=20, width=8, max_features=17){
    plt <- plot(bd, max_features=max_features)
    pdf(path, height=height, width=width)
    print(plt)
    dev.off()
}

create_cpp_plot <- function(explainer, x, variables){
    cpp <- ingredients::ceteris_paribus(explainer, x, variables = variables)
    cpp <- correct_labels_cpp(cpp)
    cpp$`_label_` <- mapvalues(cpp$`_label_`, from=c(unique(cpp$`_label_`)), to=unlist(CLASS_MAPPING, use.names = FALSE))
    cpp

}

create_pdp_plot <- function(explainer, variables){
    cpp <- ingredients::partial_dependency(explainer, variables = variables)
    cpp <- correct_labels_cpp(cpp)
    cpp$`_label_` <- mapvalues(cpp$`_label_`, from=c(unique(cpp$`_label_`)), to=unlist(CLASS_MAPPING, use.names = FALSE))
    cpp

}

In [12]:
correct_labels_cpp <- function(cpp_profile) {
  split_names_vector <- unlist(strsplit(cpp_profile$`_label_`, 'Model.'))
  cpp_profile$`_label_` <- split_names_vector[seq(2, length(split_names_vector), 2)]
  return(cpp_profile)
}

In [37]:
plot_cpp <- function(cpp_result, variable){
    plot(cpp_result, color = '_label_') +
    show_observations(cpp_result, variables = variable,
                     color='red', size=5)+
  ggtitle("Ceteris-paribus profile", "") + ylim(0, 1)+ 
labs(y='Prediction', x='Variable value') +
scale_color_manual(guide = "legend", name = "Legend", values = c("ATTM" = "#d96c06", "CTRW" = "blue", 
                                                                "FBM" = "green", "LW" = "red",
                                                                "SBM" = "black"))
}

In [17]:
save_cpp_plot <- function(path, cpp_plot, height=20, width=8){
    pdf(path, height=height, width=width)
    print(cpp_plot)
    dev.off()
}

In [59]:
create_bd_plot <- function(explainer, x){
    bd <- predict_parts(explainer, x, type='break_down', N=500)
    bd$label <- mapvalues(bd$label, from=c(unique(bd$label)), to=unlist(CLASS_MAPPING, use.names = FALSE))
    bd
}

### Predykcje na zbiorze testowym:

In [19]:
y_pred <- model$predict(x_test)

In [20]:
n_cols <- ncol(x_test)
df_predictions <- data.frame(y_real=y_test, y_pred=y_pred)
x_test_bd <- cbind(x_test, df_predictions)

In [21]:
# Real vs Pred.
common_mistakes <- list(attm_sbm=c(0, 4),
                        attm_ctrw=c(0, 1),
                        fbm_sbm=c(2, 4))

### Wybranie obserwacji, bd-plot

In [46]:
df <- data.frame()
for(el in common_mistakes){
    df_sub <- x_test_bd %>% filter(y_real==el[1],
                                   y_pred==el[2])
    df_sub <- df_sub[, 1:n_cols]
    for(row in 1:15){
        res <- create_bd_plot(explainer, df_sub[row, ])
        save_plot(os$path$join('Explanation',
                               'new',
                               'local',
                               'break_down',
                               paste0(class_vector[el[1]+1], '_', class_vector[el[2]+1]),
                                      paste0(row, '.pdf')),
                 res)
        to_spread <- res[(el[2]*(n_cols+2)+1):((el[2]+1)*(n_cols+2)), c(2, 3, 4)]
        to_add <- to_spread %>% pivot_wider(names_from='variable_name', values_from=c('contribution','variable_value'))
        to_add[, 'predicted_class'] <- el[2]
        to_add[, 'real_class'] <- el[1]
        df <- rbind(df, to_add)
        
    }
}

In [47]:
write.csv(df, os$path$join('Data', 'break-down-values-new-model.csv'))

## Ceteris paribus profile

In [49]:
df <- data.frame()
for(el in common_mistakes){
    df_sub <- x_test_bd %>% filter(y_real==el[1],
                                   y_pred==el[2])
    df_sub <- df_sub[, 1:n_cols]
    for(row in 1:15){
        print(row)
        for (variable in c('M', 'mw_x_mean_10','mw_y_mean_10',
                          'max_std_x', 'max_std_y', 'L', 'dagostino_x',
                          'mw_y_mean', 'mean_gaussianity', 'ksstat_chi2', 'p_var_1')){
            
        cpp <- create_cpp_plot(explainer, df_sub[row, ], variables = c(variable))
            res <- plot_cpp(cpp, variable)
        
        save_cpp_plot(os$path$join('Explanation',
                               'new',
                               'local',
                               'ceteris_paribus',                                   
                               paste0(class_vector[el[1]+1], '_', class_vector[el[2]+1]),
                                   variable,
                                      paste0(row, '.pdf')), cpp_plot=res,
                     width=10, height=8)
        }
    }
}

[1] 1


Scale for 'colour' is already present. Adding another scale for 'colour',
which will replace the existing scale.

Scale for 'colour' is already present. Adding another scale for 'colour',
which will replace the existing scale.

Scale for 'colour' is already present. Adding another scale for 'colour',
which will replace the existing scale.

Scale for 'colour' is already present. Adding another scale for 'colour',
which will replace the existing scale.

Scale for 'colour' is already present. Adding another scale for 'colour',
which will replace the existing scale.

Scale for 'colour' is already present. Adding another scale for 'colour',
which will replace the existing scale.

Scale for 'colour' is already present. Adding another scale for 'colour',
which will replace the existing scale.

Scale for 'colour' is already present. Adding another scale for 'colour',
which will replace the existing scale.

Scale for 'colour' is already present. Adding another scale for 'colour',
which will rep

[1] 2


Scale for 'colour' is already present. Adding another scale for 'colour',
which will replace the existing scale.

Scale for 'colour' is already present. Adding another scale for 'colour',
which will replace the existing scale.

Scale for 'colour' is already present. Adding another scale for 'colour',
which will replace the existing scale.

Scale for 'colour' is already present. Adding another scale for 'colour',
which will replace the existing scale.

Scale for 'colour' is already present. Adding another scale for 'colour',
which will replace the existing scale.

Scale for 'colour' is already present. Adding another scale for 'colour',
which will replace the existing scale.

Scale for 'colour' is already present. Adding another scale for 'colour',
which will replace the existing scale.

Scale for 'colour' is already present. Adding another scale for 'colour',
which will replace the existing scale.

Scale for 'colour' is already present. Adding another scale for 'colour',
which will rep

[1] 3


...

Scale for 'colour' is already present. Adding another scale for 'colour',
which will replace the existing scale.

Scale for 'colour' is already present. Adding another scale for 'colour',
which will replace the existing scale.

Scale for 'colour' is already present. Adding another scale for 'colour',
which will replace the existing scale.

Scale for 'colour' is already present. Adding another scale for 'colour',
which will replace the existing scale.

Scale for 'colour' is already present. Adding another scale for 'colour',
which will replace the existing scale.

Scale for 'colour' is already present. Adding another scale for 'colour',
which will replace the existing scale.

Scale for 'colour' is already present. Adding another scale for 'colour',
which will replace the existing scale.

Scale for 'colour' is already present. Adding another scale for 'colour',
which will replace the existing scale.

Scale for 'colour' is already present. Adding another scale for 'colour',
which will rep

[1] 14


Scale for 'colour' is already present. Adding another scale for 'colour',
which will replace the existing scale.

Scale for 'colour' is already present. Adding another scale for 'colour',
which will replace the existing scale.

Scale for 'colour' is already present. Adding another scale for 'colour',
which will replace the existing scale.

Scale for 'colour' is already present. Adding another scale for 'colour',
which will replace the existing scale.

Scale for 'colour' is already present. Adding another scale for 'colour',
which will replace the existing scale.

Scale for 'colour' is already present. Adding another scale for 'colour',
which will replace the existing scale.

Scale for 'colour' is already present. Adding another scale for 'colour',
which will replace the existing scale.

Scale for 'colour' is already present. Adding another scale for 'colour',
which will replace the existing scale.

Scale for 'colour' is already present. Adding another scale for 'colour',
which will rep

[1] 15


Scale for 'colour' is already present. Adding another scale for 'colour',
which will replace the existing scale.

Scale for 'colour' is already present. Adding another scale for 'colour',
which will replace the existing scale.

Scale for 'colour' is already present. Adding another scale for 'colour',
which will replace the existing scale.

Scale for 'colour' is already present. Adding another scale for 'colour',
which will replace the existing scale.

Scale for 'colour' is already present. Adding another scale for 'colour',
which will replace the existing scale.

Scale for 'colour' is already present. Adding another scale for 'colour',
which will replace the existing scale.

Scale for 'colour' is already present. Adding another scale for 'colour',
which will replace the existing scale.

Scale for 'colour' is already present. Adding another scale for 'colour',
which will replace the existing scale.

Scale for 'colour' is already present. Adding another scale for 'colour',
which will rep

## PDP

In [71]:
c(unlist(colnames(x_test)))

[1] "D"                               "alpha"                          
 [3] "alpha_n_1"                       "alpha_n_2"                      
 [5] "alpha_n_3"                       "fractal_dimension"              
 [7] "mean_gaussianity"                "mean_squared_displacement_ratio"
 [9] "straightness"                    "p-variation"                    
[11] "max_excursion_normalised"        "ksstat_chi2"                    
[13] "p_var_1"                         "p_var_2"                        
[15] "p_var_3"                         "p_var_4"                        
[17] "p_var_5"                         "dagostino_x"                    
[19] "dagostino_y"                     "M"                              
[21] "L"                               "J"                              
[23] "max_ts"                          "trappedness"                    
[25] "asymmetry"                       "diff_kurtosis"                  
[27] "efficiency"                      "mw_x_mean"                      
[29] "mw_y_mean"                       "mw_x_std"                       
[31] "mw_y_std"                        "mw_x_mean_10"                   
[33] "mw_y_mean_10"                    "mw_x_std_10"                    
[35] "mw_y_std_10"                     "max_std_x"                      
[37] "max_std_y"                       "max_std_change_x"               
[39] "max_std_change_y"                "vac_lag_1"                      
[41] "vac_lag_2"                       "dma_lag_1"                      
[43] "dma_lag_2"

In [41]:
for(el in common_mistakes){

        for (variable in unlist(colnames(x_test))){
            
        pdp <- create_pdp_plot(explainer, variables = c(variable))
            res <- plot(pdp)
        
        save_cpp_plot(os$path$join('Explanation',
                               'new',
                               'local',
                               'partial_dependency',                                   
                               paste0(class_vector[el[1]+1], '_', class_vector[el[2]+1]),
                                   
                                      paste0(variable, '.pdf')), cpp_plot=res,
                     width=10, height=8)
        }
    
}